In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
songs = pd.read_csv('songdata.csv')
songs

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


In [2]:
df=songs

In [3]:
df=df[df['song']=='Only Over You']
t=df['text'].tolist()

In [4]:
print(t)

["I'm out of my mind  \nAnd it's only over you  \nI'm out of my mind  \nAnd it's only over you  \nPeople think I'm crazy  \n  \nBut they don't know  \nThought love had failed me  \nBut now, they're watching it grow  \nAngel, please don't go  \nI miss you when you're gone  \nThey say I'm a silly girl  \n  \nBut I'm not a fool  \nPeople say they know me  \nBut they don't see  \nMy heart's your future  \nYour future is me  \nAngel, please don't go  \nI miss you when you're gone  \nThey say I'm a silly girl  \n  \nBut I'm not a fool  \nI'm out of my mind  \nAnd it's only over you  \nI'm out of my mind  \nAnd it's only over you\n\n"]


In [5]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)
songs['text'] = songs['text'].str.replace(r'\n', '')
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf.fit_transform(songs['text'])

C:\Users\HP\AppData\Local\Temp\ipykernel_5492\2994419509.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


In [6]:
tfidf_matrix

<5000x23812 sparse matrix of type '<class 'numpy.float64'>'
	with 267073 stored elements in Compressed Sparse Row format>

In [7]:
import numpy as np
from scipy.sparse import csr_matrix

# Get the array of values stored in the matrix
values = tfidf_matrix.data

# Get the array of column indices for the values
indices = tfidf_matrix.indices
count=0
# Iterate over the values and indices, and print the values
for value, index in zip(values, indices):
    print(value, index)
    count+=1
    if(count==10):
        break

0.10094461357590333 12377
0.10927894959554575 6329
0.11922980861123222 12123
0.08859040081214907 17729
0.08970277116981348 951
0.08465485461175287 9275
0.07658433822755903 1465
0.05909433339530094 8761
0.06831492716968758 12036
0.09457314295883053 11713


In [8]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(tfidf_matrix) 
similarities = {}
for i in range(len(cosine_similarities)):
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], 
                                           songs['artist'][x]) for x in similar_indices][1:]

In [9]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix
    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)       
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------") 
    def recommend(self, recommendation):
            song = recommendation['song']
            number_songs = recommendation['number_songs']
            recom_song = self.matrix_similar[song][:number_songs]
            self._print_message(song=song, recom_song=recom_song)       

In [10]:
recommedations = ContentBasedRecommender(similarities)


recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 5
}

recommedations.recommend(recommendation)

The 5 recommended songs for Get At Ya are:
Number 1:
Can You Help Me by Usher with 0.288 similarity score
--------------------
Number 2:
I Really Like You by Pharrell Williams with 0.283 similarity score
--------------------
Number 3:
How'd Ya Do by Eddie Cochran with 0.281 similarity score
--------------------
Number 4:
Why You Up In Here by Flo-Rida with 0.272 similarity score
--------------------
Number 5:
Hey Little Girl by Elvis Presley with 0.241 similarity score
--------------------


In [13]:
#you can provide the name of a song and get recommendation too(provided it is in the dataset)
recommendation2 = {
    "song": 'Can You Help Me',
    "number_songs": 4 
}
recommedations.recommend(recommendation2)

The 4 recommended songs for Can You Help Me are:
Number 1:
Why Do I Keep Counting? by The Killers with 0.38 similarity score
--------------------
Number 2:
How'd Ya Do by Eddie Cochran with 0.369 similarity score
--------------------
Number 3:
Heaven Help Us All by Ray Charles with 0.361 similarity score
--------------------
Number 4:
The Secret Garden by Quincy Jones with 0.359 similarity score
--------------------
